In [1]:
""" This File is our baseline tool for getting the Open, High, Low, Close, Volume for BTC.
    It uses Binance API and creates DataFrames for further data manipulation and TA.
    We will use this for the Logistic Regression."""
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd
import mplfinance as mpl
import os
import numpy as np
from hvplot import hvPlot
import pandas as pd
from ta import add_all_ta_features
from ta.utils import dropna
import pandas_ta as ta
from finta import TA

In [ ]:
pip install pandas_ta

In [ ]:
pip install --upgrade ta

In [ ]:
api_key = os.environ.get("api_key")
api_secret = os.environ.get("api_secret")

In [ ]:
client = Client(api_key, api_secret)

In [ ]:
tickers = client.get_all_tickers()

In [ ]:
#tickers()

In [ ]:
#tickers[1]['price']


In [ ]:
ticker_df = pd.DataFrame(tickers)

In [ ]:
#ticker_df.head()

In [ ]:
#ticker_df.tail()

In [ ]:
ticker_df.set_index('symbol', inplace=True)

In [ ]:
float(ticker_df.loc['ETHUSDC']['price'])
float(ticker_df.loc['BTCUSDT']['price'])

In [ ]:
# Get Depth
depth = client.get_order_book(symbol='BTCUSDT')

In [ ]:
# depth

In [ ]:
depth_df = pd.DataFrame(depth['asks'])
depth_df.columns = ['Price', 'Volume']
depth_df.head()

In [ ]:
depth_df.dtypes

In [ ]:
# client.get_historical_klines()

In [ ]:
# BTC Minute Data from past day
historical_df = client.get_historical_klines('BTCUSDC', Client.KLINE_INTERVAL_1MINUTE, '1 day ago UTC')

In [ ]:
hist_df = pd.DataFrame(historical_df)

In [ ]:
# Function to get list of OHLCV Values
#def getminutedata(symbol, interval, lookback)

In [ ]:
hist_df.head()

In [ ]:
hist_df.columns = ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume', 
                    'Number of Trades', 'TB Base Volume', 'TB Quote Volume', 'Ignore']

In [ ]:
hist_df.tail()

In [ ]:
hist_df.shape

In [ ]:
hist_df.dtypes

In [ ]:
hist_df['Open Time'] = pd.to_datetime(hist_df['Open Time']/1000, unit='s')
hist_df['Close Time'] = pd.to_datetime(hist_df['Close Time']/1000, unit='s')

In [ ]:
numeric_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote Asset Volume', 'TB Base Volume', 'TB Quote Volume']

In [ ]:
hist_df[numeric_columns] = hist_df[numeric_columns].apply(pd.to_numeric, axis=1)

In [ ]:
hist_df.tail()

In [ ]:
hist_df.dtypes


In [ ]:
hist_df.describe()


In [ ]:
hist_df.info()

In [ ]:
# Technical Analysis
import mplfinance as mpf

In [ ]:
hist_df.set_index('Close Time').tail(100)

In [ ]:
mpf.plot(hist_df.set_index('Close Time').tail(50), 
        type='candle', style='charles', 
        volume=True, 
        title='BTCUSDC Last 5 Hours', 
        mav=(10,20,30))

In [ ]:
""" Daily Data from 2021 
use the client.get_historical_klines function and input symbol, interval, and start date.
End date is left out becaues we want current data """

btc_daily_data = client.get_historical_klines('BTCUSDC', Client.KLINE_INTERVAL_1DAY, '1 Jan 2021')

In [ ]:
btc_daily_df = pd.DataFrame(btc_daily_data)

In [ ]:
btc_daily_df.columns = ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume', 
                    'Number of Trades', 'TB Base Volume', 'TB Quote Volume', 'Ignore']

In [ ]:
btc_daily_df['Open Time'] = pd.to_datetime(btc_daily_df['Open Time']/1000, unit='s')
btc_daily_df['Close Time'] = pd.to_datetime(btc_daily_df['Close Time']/1000, unit='s')

In [ ]:
numeric_columns_d = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote Asset Volume', 'TB Base Volume', 'TB Quote Volume']

In [ ]:
btc_daily_df[numeric_columns] = btc_daily_df[numeric_columns].apply(pd.to_numeric, axis=1)

In [ ]:
btc_daily_df.describe()

In [ ]:
mpf.plot(btc_daily_df.set_index('Close Time').tail(284), 
        type='candle', style='charles', 
        volume=True, 
        title='BTCUSDC 2021 Daily ', 
        mav=(10,50,100))

In [ ]:
# Just get the Open Time, Open, High, Low, Close, Volume
btc_ohlcv_daily = btc_daily_df.iloc[:,0:6]
btc_ohlcv_minute = hist_df.iloc[:,0:6]

In [ ]:
# CSV SAVER TOOL For Saving Bitcoin data to CSVs to use for LSTM
btc_ohlcv_daily.to_csv('Data/btc_daily_lstm', index=False)
btc_ohlcv_minute.to_csv('Data/btc_minutelstm', index=False)

In [ ]:
btc_ohlcv_daily = btc_ohlcv_daily.set_index('Open Time')
btc_ohlcv_minute = btc_ohlcv_minute.set_index('Open Time')

In [ ]:
# Display of Daily Bitcoin DF to use for TA and Machine Leaning
display(btc_ohlcv_daily.head())
display(btc_ohlcv_minute.tail())

In [ ]:
""" Momentum Indicators for Bitcoin Minute Data
make target column and features X"""

In [ ]:
# TA for btc minute data
short_window=5
long_window=50
# get the actual minute % returns
btc_ohlcv_minute["actual returns"] = btc_ohlcv_minute["Close"].pct_change().dropna()
# Simple Moving Average(SMA)
btc_ohlcv_minute['sma5'] = ta.sma(btc_ohlcv_minute["Close"], short_window)
btc_ohlcv_minute['sma50'] = ta.sma(btc_ohlcv_minute["Close"], long_window)
# Additional Technical Indicators
# btc_ohlcv_minute['ssma'] = TA.SSMA(btc_ohlcv_minute)
# btc_ohlcv_minute['ema'] = TA.EMA(btc_ohlcv_minute, 50)
# btc_ohlcv_minute['dema'] = TA.DEMA(btc_ohlcv_minute)
# btc_ohlcv_minute['tema'] = TA.TEMA(btc_ohlcv_minute)
# btc_ohlcv_minute['trima'] = TA.TRIMA(btc_ohlcv_minute)
btc_ohlcv_minute.dropna()

In [ ]:
# Create a column to hold the trading signal 
btc_ohlcv_minute['Signal'] = 0.0

"""First Simple Strategy for Longing a Bullish Market
# Initializing signal column for either longing or shorting. 1 is when the short window SMA is greated than the Long Window SMA
# 1 is short window > long, whereas 0 is when condition is not met 
"""
btc_ohlcv_minute["Signal"][short_window:] = np.where(
    btc_ohlcv_minute['sma5'][short_window:] > btc_ohlcv_minute['sma50'][short_window:], 1.0, 0.0
)

# Find the CrossOver points using Diff()
# Calculate the points in time when the Signal value changes
# Identify trade entry(1) and exit (-1)
# btc_ohlcv_minute["Entry/Exit"] = btc_ohlcv_minute["Signal"].diff()


# REVIEW SIGNAL DF
btc_ohlcv_minute

In [ ]:
# # Visualize exit position relative to close price
# exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Close'].hvplot.scatter(
#     color=’yellow',
#     legend=False,
#     ylabel='Price in $',
#     width=1000,
#     height=400)

# # Show the plot
# exit

# # Visualize entry position relative to close price
# entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Close'].hvplot.scatter(
#     color='purple',
#     legend=False,
#     ylabel='Price in $',
#     width=1000,
#     height=400)

# # Show the plot
# entry

# # Visualize close price for the investment
# security_close = signals_df[['Close']].hvplot(
#     line_color='lightgray',
#     ylabel='Price in $',
#     width=1000,
#     height=400)

# # Show the plot
# security_close

# # Visualize moving averages
# moving_avgs = signals_df[['SMA50', 'SMA100']].hvplot(
#     ylabel='Price in $',
#     width=1000,
#     height=400)

# # Show the plot
# moving_avgs

In [ ]:
# RUn Back Testing

In [ ]:
btc_ohlcv_minute.columns

In [ ]:
# Create target and features columns
y = btc_ohlcv_minute['Signal']
X = btc_ohlcv_minute[['sma5', 'sma50']].shift().dropna().copy()

In [ ]:
# CSV SAVER TOOL For Saving Bitcoin data to CSVs to use for Logisitc Regression model
# btc_ohlcv_daily.to_csv('daily_btc_ohclv_2021', index=False)
# btc_ohlcv_minute.to_csv('minute_btc_ohclv_5th_6th', index=False)

In [ ]:
pip install --upgrade ta

In [ ]:
pip install pandas_ta

In [ ]:
import pandas as pd
from ta import add_all_ta_features
from ta.utils import dropna
import pandas_ta as ta
from finta import TA

In [ ]:
""" Trend Indicators for Bitcoin Daily Data
make target column and features X"""

In [ ]:
# TA for BTC Daily Data
short_window=5
long_window=50
# Daily Returns
btc_ohlcv_daily["actual returns"] = btc_ohlcv_daily["Close"].pct_change()
btc_ohlcv_daily.dropna()
# Simple Moving Average(SMA) short_window = 5, long window = 50
btc_ohlcv_daily['sma5'] = ta.sma(btc_ohlcv_daily["Close"], short_window)
btc_ohlcv_daily['sma50'] = ta.sma(btc_ohlcv_daily["Close"], long_window)
# Additional Technical Indicators
# btc_ohlcv_daily['ssma'] = TA.SSMA(btc_ohlcv_daily)
# btc_ohlcv_daily['ema'] = TA.EMA(btc_ohlcv_daily, 50)
# btc_ohlcv_daily['dema'] = TA.DEMA(btc_ohlcv_daily)
# btc_ohlcv_daily['tema'] = TA.TEMA(btc_ohlcv_daily)
# btc_ohlcv_daily['trima'] = TA.TRIMA(btc_ohlcv_daily)
btc_ohlcv_daily.dropna()

In [ ]:
# Create a column to hold the trading signal 
btc_ohlcv_daily['Signal'] = 0.0

"""First Simple Strategy for Longing a Bullish Market
# Initializing signal column for either longing or shorting. 1 is when the short window SMA is greated than the Long Window SMA
# 1 is short window > long, whereas 0 is when condition is not met 
"""
btc_ohlcv_daily["Signal"][short_window:] = np.where(
    btc_ohlcv_daily['sma5'][short_window:] > btc_ohlcv_daily['sma50'][short_window:], 1.0, 0.0
)

# Find the CrossOver points using Diff()
# Calculate the points in time when the Signal value changes
# Identify trade entry(1) and exit (-1)
# btc_ohlcv_daily["Entry/Exit"] = btc_ohlcv_daily["Signal"].diff()


# REVIEW SIGNAL DF
btc_ohlcv_daily

In [ ]:
# Create target and features columns
y = btc_ohlcv_daily['Signal']
X = btc_ohlcv_daily[['sma5', 'sma50']].shift().dropna().copy()

In [ ]:
btc_ohlcv_daily.head()

In [ ]:
# CSV SAVER TOOL For Saving Bitcoin data to CSVs to use for Logisitc Regression model
btc_ohlcv_daily.to_csv('Data/daily_btc_logreg', index=False)
btc_ohlcv_minute.to_csv('Data/min_btc_logreg', index=False)